In [1]:
import pygad 
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal, skew
from numba import int32, float64   # import the types
from portfolio_simulator import PortfoliosSimulator

In [2]:
def expected_returns(period):
    return pd.Series({
		'ACWI': 0.07/period,
		'^TBSP': 0.035/period,
		'XAUPLN': 0.05/period,
		'MWIG40': 0.065/period,
		'IHYU.L': 0.038/period,
		'PLOPLN3M': 0.03/period,
		'edo': 0.032/period
})

""" cov = np.array([[0.001265,	0.000042],
            	[0.000042,	0.000164]]) """
#acwi edo
cov = np.array([[0.030276,	0.0],
            	[0.0,	0.00021]])
cov_y = cov 

assets = ['ACWI', 'edo']
means = expected_returns(1)[assets]
number_of_scenarios = 10000
T = 50
sim_returns = []

for s in range (number_of_scenarios):
    sim_returns.append(multivariate_normal.rvs(means,cov_y, T))

sim_returns = np.array(sim_returns)
sim_returns.shape

(10000, 50, 2)

In [3]:
class Simulation():
    def __init__(self, initial_value, inflows, returns, strategy) -> None:
        self.capital = initial_value
        self.returns = returns
        self.strategy = strategy 
        self.inflows = inflows
        
    def step(self, current_value,inflow,weights,returns):
        current_assets_value = np.expand_dims(current_value+inflow,axis=0).T * weights
        next_assets_value =  np.round(current_assets_value * (1+returns),2)
        return next_assets_value.sum(1)
         
    def run(self, T):
        for t in range(T):
            self.capital = self.step(self.capital,self.inflows[t],self.strategy[t],self.returns[:,t])

In [5]:
portfolios = [[0.0,1.0],
             [0.1,0.9],
             [0.2,0.8],
             [0.3,0.7],
             [0.4,0.6],
             [0.5,0.5],
             [0.6,0.4],
             [0.7,0.3],
             [0.8,0.2],
             [0.9,0.1],
             [1.0,0.0]] 

In [6]:
T = 50
capital = 50000
inflows = np.cumproduct(np.full(T,1.02))*26000

l = np.arange(T,0,-1)
risk = 10

glide_paths = np.full(T+1,len(portfolios))
perc = np.zeros((l.shape[0],len(portfolios)))

for t in l:
    
    for portfolio in range(len(portfolios)):
        solution = np.full(T,portfolio)
        strategy = np.take(portfolios,solution, axis=0)
        simulator = Simulation(capital,inflows, sim_returns,strategy)
        simulator.run(t)
        #perc.append(np.percentile(simulator.capital,risk))
        perc[t-1,portfolio] = np.round(np.percentile(simulator.capital,risk))
    glide_paths[t-1] = np.argmax(perc[t-1])

#glide = list(glide_paths.values())
np.flip(glide_paths)[1:]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0])

In [11]:
from typing import Any

class MaxPercentile():

    def __init__(self, percentile) -> None:
        self.percentile = percentile

    def __call__(self, values: np.array) -> float:
        return np.percentile(values,self.percentile)

class Fitness:
    def __init__(self,simulator,solution,objective) -> None:
        self.simulator = simulator
        self.solution = solution
        self.objective

    def __call__(self, *args: Any, **kwds: Any) -> Any:
        pass

In [14]:
objective = MaxPercentile(50)

objective(np.arange(1000))

499.5

In [7]:
T = 50
capital = 50000
inflows = np.cumproduct(np.full(T,1.02))*26000

l = np.arange(T,0,-1)
risk = 10

glide_paths = np.full(T+1,len(portfolios))
perc = np.zeros((l.shape[0],len(portfolios)))

for t in l:
    
    for portfolio in range(len(portfolios)):
        solution = np.full(T,portfolio)
        strategy = np.take(portfolios,solution, axis=0)
        simulator = Simulation(capital,inflows, sim_returns,strategy)
        simulator.run(t)
        #perc.append(np.percentile(simulator.capital,risk))
        perc[t-1,portfolio] = np.round(np.percentile(simulator.capital,risk))
    glide_paths[t-1] = np.argmax(perc[t-1])

#glide = list(glide_paths.values())
np.flip(glide_paths)[1:]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0])